In [2]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast
from fuzzywuzzy import process
from nltk.corpus import words

## Initial Experiments

In [ ]:
total = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\total_summary_stats_updated.csv")

In [ ]:
patient_id = total["patient_id"].to_list()
patient_id = patient_id[:10]

In [ ]:
matches = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_matches.csv")

In [ ]:
# ONLY RUN ONCE #

# vertifying that all match records can be found
for i in range(len(patient_id)):
    specific_matches = matches[matches["PatientID"] == patient_id[i]]
    if (len(specific_matches) <= 0):
        print("NOT FOUND: ", patient_id[i])
    #specific_matches.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Specific_Matches\patient_{}.csv".format(i+1))

In [ ]:
p1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Specific_Matches\patient_1.csv")

In [ ]:
unique_ids = matches["PatientID"].unique().tolist()
print(len(unique_ids))

In [ ]:
def get_match(query, choices, limit = 1):
    results = process.extract(query, choices, limit = limit)
    return results

In [ ]:
# interesting match cases
# patient_2 in specific study row 6, cream that has mci inside it

In [ ]:
subset_loc = p1[p1["PatientID"] == 'Z6352398']['regex_location'].to_list()
subset_loc_clone = p1[p1["PatientID"] == 'Z6352398']['regex_location'].to_list()
#print(subset_loc)
encounters = p1[p1["PatientID"] == 'Z6352398']['PatientEncounterID'].unique().tolist()

#print(len(subset_loc))
#print(subset_loc)
#print(subset_loc[0])
for i in range(len(subset_loc)):
    subset_loc[i] = subset_loc[i][1:len(subset_loc[i]) - 1]
    subset_loc[i] = eval(subset_loc[i])
    #subset_loc[i] = tuple(map(int, subset_loc[i].split(", ")))
    subset_loc_clone[i] = subset_loc_clone[i][1:len(subset_loc_clone[i]) - 1]
    subset_loc_clone[i] = eval(subset_loc_clone[i])
    
subset_loc = pd.Series(subset_loc)
subset_loc = subset_loc.unique().tolist()

subset_loc_clone = pd.Series(subset_loc_clone)
subset_loc_clone = subset_loc_clone.unique().tolist()

print("CLONE: ", subset_loc_clone)
print(subset_loc)
print("\n", subset_loc[0][0])
print(len(subset_loc))
if (len(subset_loc) != len(encounters)):
    print("Houston, we got a problem", i, len(subset_loc), len(encounters))

total_merged = []  

for i in range(len(subset_loc)):
    merged = []

    start = 0
    end = 0
    length = 0
    
    if (len(subset_loc[i]) == 2):
        start = subset_loc[i][0]
        end = subset_loc[i][1]
        length = len(p1[(p1["regex_location"] == "[" +str(subset_loc[i]) + "]") & (p1["PatientEncounterID"] == int(encounters[i]))]['NoteTXT'].values[0])
        if (start < 100):
            start = 100
        elif (end > length - 100):
             end = length - 100
        subset_loc[i] = [(start - 100, end + 100)]
        
    else:
        loc_list = []
        for j in range(len(subset_loc[i])):
            start = subset_loc[i][j][0]
            end = subset_loc[i][j][1]
            loc_list.append((start, end))
        length = len(p1[(p1["regex_location"] == str(loc_list)) & (p1["PatientEncounterID"] == int(encounters[i]))]['NoteTXT'].values[1])
        for k in range(len(loc_list)):
            start = loc_list[k][0]
            end = loc_list[k][1]
            if (start < 100):
                start = 100
            elif (end > length - 100):
                end = length - 100
            loc_list[k] = (start - 100, end + 100)
        subset_loc[i] = loc_list
    
    subset_loc[i].sort(key = lambda x : x[0])
    
    print("\nSUBSET_LOC", subset_loc[i])
    
    for interval in subset_loc[i]:
        if not merged:
            merged.append(interval)
            continue

        prevInterval = merged.pop()
        if prevInterval[0] <= interval[0] <= prevInterval[1]:
            startTime = prevInterval[0]
            endTime = prevInterval[1] if prevInterval[1] > interval[1] else interval[1]
            merged.append((startTime, endTime))
        else:
            merged.append(prevInterval)
            merged.append(interval)
    total_merged.append(merged)
    
total_merged
#     print("start", start)
#     print("end", end)
    #print("matches, ", p1[(p1["regex_location"] == "[" +str(subset_loc[j]) + "]") & (p1["PatientEncounterID"] == int(encounters[j]))]['NoteTXT'].values)
    #print("length, ", length) 
    

In [ ]:
print(subset_loc_clone)

In [ ]:
total_note_txt = []
print(len(total_merged))
for i in range(len(total_merged)):
    curr = ""
    if (len(total_merged[i]) == 1):
        note = p1[(p1["regex_location"] == "[" +str(subset_loc_clone[i]) + "]") & (p1["PatientEncounterID"] == int(encounters[i]))]['NoteTXT'].values[0]
        #print(note)
        #print(total_merged[i][0][0], total_merged[i][0][1])
        curr = note[total_merged[i][0][0]:total_merged[i][0][1]]
    else:
        for j in range(len())
    total_note_txt.append(curr)
    
total_note_txt

In [ ]:
df = pd.DataFrame(total_note_txt, columns = ["text"])
df = df.drop_duplicates(subset='text', keep="first")
df = df.replace(r'^\s*$', np.NaN, regex=True)
df.dropna()
total_note_txt = df["text"].to_list()

In [ ]:
total_note_txt

## Final Code

In [48]:
p1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\5\apoe_5_matches.csv")

In [49]:
p1.head()

,Unnamed: 0,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count
0,0,106692571,2871964,Z9527072,3.163956e+09,2017-07-17 00:00:00.0000000,Hospital Encounter,1593168404,Discharge Summary,5,1.539980e+09,1.0,----------------------------------------------...,"[(7211, 7220)]","""Neuro: Alert and oriented, no focal deficits ...","[""Cognition""]",1
1,1,106693931,10046146527,Z9535446,3.152680e+09,2017-04-27 00:00:00.0000000,Office Visit,1494524615,Progress Notes,5,1.438352e+09,1.0,----------------------------------------------...,"[(2955, 2961)]","""rashes, lesions or ulcers. Mental Status: Ale...","[""memory""]",1
2,2,106693931,10046146527,Z9535446,3.152680e+09,2017-04-27 00:00:00.0000000,Office Visit,1514654085,Letter,5,1.459068e+09,1.0,----------------------------------------------...,"[(3477, 3483)]","""rashes, lesions or ulcers. Mental Status: Ale...","[""memory""]",1
3,3,106693931,10046146527,Z9535446,3.181874e+09,2017-12-09 00:00:00.0000000,Hospital Encounter,1783311709,ED Provider Notes,5,1.735375e+09,1.0,----------------------------------------------...,"[(8155, 8164)]","""Discharged: Home Condition on Discharge: Sati...","[""cognition""]",1
4,4,106693931,10046146527,Z9535446,3.294365e+09,2020-03-02 00:00:00.0000000,Office Visit,3773193065,Progress Notes,5,3.754601e+09,1.0,----------------------------------------------...,"[(7796, 7805)]","""PM&R Sports Medicine Newton-Wellesley Spine *...","[""cognition""]",1


In [50]:
total_merged = []

for index, row in p1.iterrows():
    curr = str(row["regex_location"])
    curr = curr[1:len(curr) - 1]
    curr = list((eval(curr)))
    #print("CURR: ", len(curr))
    # print(curr)
    merged = []

    start = 0
    end = 0
    length = 0
    
    if (len(curr) == 2 and isinstance(curr[1], int)):
        start = curr[0]
        end = curr[1]
        #print(start, end)
        length = len(row["NoteTXT"])
        if (start < 100):
            start = 100
        elif (end > length - 100):
            end = length - 100
        curr = [(start - 100, end + 100)]
        
    else:
        loc_list = []
        for j in range(len(curr)):
            start = curr[j][0]
            end = curr[j][1]
            loc_list.append((start, end))
        length = len(row["NoteTXT"])
        for k in range(len(loc_list)):
            start = loc_list[k][0]
            end = loc_list[k][1]
            if (start < 100):
                start = 100
            elif (end > length - 100):
                end = length - 100
            loc_list[k] = (start - 100, end + 100)
        curr = loc_list
        
    for interval in curr:
        if not merged:
            merged.append(interval)
            continue

        prevInterval = merged.pop()
        if prevInterval[0] <= interval[0] <= prevInterval[1]:
            startTime = prevInterval[0]
            endTime = prevInterval[1] if prevInterval[1] > interval[1] else interval[1]
            merged.append((startTime, endTime))
        else:
            merged.append(prevInterval)
            merged.append(interval)
            
    total_merged.append(merged)
p1["merged_row_location"] = total_merged

In [51]:
p1

,Unnamed: 0,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count,merged_row_location
0,0,106692571,2871964,Z9527072,3.163956e+09,2017-07-17 00:00:00.0000000,Hospital Encounter,1593168404,Discharge Summary,5,1.539980e+09,1.0,----------------------------------------------...,"[(7211, 7220)]","""Neuro: Alert and oriented, no focal deficits ...","[""Cognition""]",1,"[(7111, 7320)]"
1,1,106693931,10046146527,Z9535446,3.152680e+09,2017-04-27 00:00:00.0000000,Office Visit,1494524615,Progress Notes,5,1.438352e+09,1.0,----------------------------------------------...,"[(2955, 2961)]","""rashes, lesions or ulcers. Mental Status: Ale...","[""memory""]",1,"[(2855, 3061)]"
2,2,106693931,10046146527,Z9535446,3.152680e+09,2017-04-27 00:00:00.0000000,Office Visit,1514654085,Letter,5,1.459068e+09,1.0,----------------------------------------------...,"[(3477, 3483)]","""rashes, lesions or ulcers. Mental Status: Ale...","[""memory""]",1,"[(3377, 3583)]"
3,3,106693931,10046146527,Z9535446,3.181874e+09,2017-12-09 00:00:00.0000000,Hospital Encounter,1783311709,ED Provider Notes,5,1.735375e+09,1.0,----------------------------------------------...,"[(8155, 8164)]","""Discharged: Home Condition on Discharge: Sati...","[""cognition""]",1,"[(8055, 8264)]"
4,4,106693931,10046146527,Z9535446,3.294365e+09,2020-03-02 00:00:00.0000000,Office Visit,3773193065,Progress Notes,5,3.754601e+09,1.0,----------------------------------------------...,"[(7796, 7805)]","""PM&R Sports Medicine Newton-Wellesley Spine *...","[""cognition""]",1,"[(7696, 7905)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278148,278163,113722742,10127721883,Z16337903,3.329813e+09,2020-11-04 00:00:00.0000000,Office Visit,4806472815,Progress Notes,5,4.806423e+09,1.0,----------------------------------------------...,"[(3680, 3689)]","""lesions. Normal turgor. Psych: Alert and orie...","[""cognition""]",1,"[(3580, 3789)]"
278149,278164,113722742,10127721883,Z16337903,3.335051e+09,2021-06-17 00:00:00.0000000,Office Visit,5986742692,Progress Notes,5,5.985886e+09,1.0,----------------------------------------------...,"[(12478, 12487)]","""documentation, and care coordination. This do...","[""cognition""]",1,"[(12378, 12587)]"
278150,278165,113722742,10127721883,Z16337903,3.358959e+09,2021-03-17 00:00:00.0000000,Telemedicine,5484338769,Progress Notes,5,5.485571e+09,1.0,----------------------------------------------...,"[(5455, 5464)]","""and coordination, normal speech. Psych: Behav...","[""cognition""]",1,"[(5355, 5564)]"
278151,278166,113722742,10127721883,Z16337903,3.362169e+09,2021-04-22 00:00:00.0000000,Office Visit,5695272756,Progress Notes,5,5.778178e+09,1.0,----------------------------------------------...,"[(7678, 7687)]","""Awake and oriented, normal speech. Psych: Beh...","[""cognition""]",1,"[(7578, 7787)]"


In [52]:
total_merged_note_txt = []
for index, row in p1.iterrows():
    curr = row["merged_row_location"]
    merged_note_txt = ""
    
    for i in range(len(curr)):
        start = curr[i][0]
        end = curr[i][1]
        
        merged_note_txt += " ------- " + row["NoteTXT"][start:end]
    total_merged_note_txt.append(merged_note_txt)
    
p1["merged_regex_sent"] = total_merged_note_txt

In [53]:
len(p1)

278153

In [54]:
p1['merged_regex_sent'] = p1['merged_regex_sent'].str.split(' ',1).str[1].str.rpartition(' ')[0]

In [55]:
#p1['NoteTXT'] = p1['NoteTXT'].astype(str)
p1 = p1.drop_duplicates(subset='merged_regex_sent', keep="first")
p1 = p1.replace(r'^\s*$', np.NaN, regex=True)
p1.reset_index(drop = True)

,Unnamed: 0,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count,merged_row_location,merged_regex_sent
0,0,106692571,2871964,Z9527072,3.163956e+09,2017-07-17 00:00:00.0000000,Hospital Encounter,1593168404,Discharge Summary,5,1.539980e+09,1.0,----------------------------------------------...,"[(7211, 7220)]","""Neuro: Alert and oriented, no focal deficits ...","[""Cognition""]",1,"[(7111, 7320)]","------- ma Neuro: Alert and oriented, no focal..."
1,1,106693931,10046146527,Z9535446,3.152680e+09,2017-04-27 00:00:00.0000000,Office Visit,1494524615,Progress Notes,5,1.438352e+09,1.0,----------------------------------------------...,"[(2955, 2961)]","""rashes, lesions or ulcers. Mental Status: Ale...","[""memory""]",1,"[(2855, 3061)]","------- apparent rashes, lesions or ulcers. Me..."
2,3,106693931,10046146527,Z9535446,3.181874e+09,2017-12-09 00:00:00.0000000,Hospital Encounter,1783311709,ED Provider Notes,5,1.735375e+09,1.0,----------------------------------------------...,"[(8155, 8164)]","""Discharged: Home Condition on Discharge: Sati...","[""cognition""]",1,"[(8055, 8264)]",------- sition Discharged: Home Condition on D...
3,4,106693931,10046146527,Z9535446,3.294365e+09,2020-03-02 00:00:00.0000000,Office Visit,3773193065,Progress Notes,5,3.754601e+09,1.0,----------------------------------------------...,"[(7796, 7805)]","""PM&R Sports Medicine Newton-Wellesley Spine *...","[""cognition""]",1,"[(7696, 7905)]",------- rd/Spaulding PM&R Sports Medicine Newt...
4,6,106693931,10046146527,Z9535446,3.317045e+09,2020-11-09 00:00:00.0000000,Office Visit,4829960783,Progress Notes,5,4.820178e+09,1.0,----------------------------------------------...,"[(6902, 6908)]","""No edema. Feet are warm and distal pulses are...","[""Memory""]",1,"[(6802, 7008)]",------- emities: No edema. Feet are warm and d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79024,278154,113722742,10127721883,Z16337903,3.223494e+09,2018-10-31 00:00:00.0000000,Office Visit,2198013886,Progress Notes,5,2.161922e+09,1.0,----------------------------------------------...,"[(2082, 2091)]","""(R) gait status, as well as getting up and do...","[""Cognition""]",1,"[(1982, 2191)]","------- (R) gait status, as well as getting u..."
79025,278155,113722742,10127721883,Z16337903,3.223896e+09,2018-11-01 00:00:00.0000000,Anesthesia Event,2199872109,Anesthesia Preprocedure Evaluation,5,2.170262e+09,1.0,----------------------------------------------...,"[(950, 958)]","""(+) Chronic female pelvic pain (On methadone ...","[""cerebral""]",1,"[(850, 1058)]",------- R/SKELETAL (+) Chronic female pelvic p...
79026,278163,113722742,10127721883,Z16337903,3.329813e+09,2020-11-04 00:00:00.0000000,Office Visit,4806472815,Progress Notes,5,4.806423e+09,1.0,----------------------------------------------...,"[(3680, 3689)]","""lesions. Normal turgor. Psych: Alert and orie...","[""cognition""]",1,"[(3580, 3789)]",------- her lesions. Normal turgor. [Psych]: A...
79027,278164,113722742,10127721883,Z16337903,3.335051e+09,2021-06-17 00:00:00.0000000,Office Visit,5986742692,Progress Notes,5,5.985886e+09,1.0,----------------------------------------------...,"[(12478, 12487)]","""documentation, and care coordination. This do...","[""cognition""]",1,"[(12378, 12587)]","------- eview, documentation, and care coordin..."


In [56]:
p1.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\5\apoe_5_matches_updated.csv", index = False)

## Tests

In [ ]:
x = ((7509, 7518), (7735, 7741), (7745, 7754), (7858, 7862))
y = []
for i in range(len(x)):
    a = x[i][0]
    b = x[i][1]
    y.append((a,b))
print(str(y))
z = len(p1[(p1["regex_location"] == str(y)) & (p1["PatientEncounterID"] == int(encounters[21]))]['NoteTXT'].values[1])
z

In [ ]:
x = [(181, 390),
  (479, 688),
  (2048, 2269),
  (6184, 6390),
  (6584, 6821),
  (7450, 7659),
  (7716, 7932),
  (9864, 10072),
  (11192, 11648),
  (12118, 12627)]
y = []
for i in range(len(x)):
    a = x[i][0]
    b = x[i][1]
    y.append((a,b))
print(y)

In [ ]:
len(p1[(p1["regex_location"] == "[" +str(subset_loc[1]) + "]") & (p1["PatientEncounterID"] == int(encounters[1]))]['NoteTXT'].values[0])

In [ ]:
print(subset_loc_clone)